In [1]:
import pandas as pd
import numpy as np
path = ''
import gc
#import xgboost as xgb
#from sklearn.cross_validation import train_test_split 
from sklearn.metrics import mean_squared_error as rmse
from sklearn.cross_validation import KFold
import sys
#sys.path.append(path+'/src/python/')
from data_load import loadHDF_train
from data_load import loadHDF_test
from time import time
from sys import stdout
from copy import deepcopy

seed = 3555
np.random.seed(seed)

kfold=5
params = {"seed": np.random.randint(0,1000),
          "eta": 0.1,  
          "objective":"reg:linear", 
          "max_depth":3, "subsample":0.3, "colsample_bylevel": 0.5, "colsample_bytree": 0.5, 
          "eval_metric": "rmse", 
          "min_child_weight": 1000, 'tree_method':'exact',
          "booster":'gbtree', 'rate_drop': 0.5, 'silent':1}
early_stop = 50
num_round = 200
clip = [1.,50.]
unit = 3600.*24.

In [2]:
outputfilehead = 'xgb_dM1dVal24-1hrAgg.CV5.2'
finalsubmitfile = 'results/'+'0905-1_'+outputfilehead+'.csv'
datafilehead = path+'feat_dM1dVal24-1hr'

print(params)

colnames = [u'timestamp', u'meanInv.custid', u'q75Inv.custid', u'q95Inv.custid',
            u'q999Inv.custid', u'rInvGT10.custid', u'rInvGT2.custid',
            u'meanInv.custidHr', u'q75Inv.custidHr', u'q95Inv.custidHr',
            u'q999Inv.custidHr', u'rInvGT10.custidHr', u'rInvGT2.custidHr',
            u'meanInv.grpidHr', u'q75Inv.grpidHr', u'q95Inv.grpidHr', 
            u'q999Inv.grpidHr', u'rInvGT10.grpidHr', u'rInvGT2.grpidHr']
#didn't use
month=6
timelimit = 3600.*24* np.array([(month-1),month])*30 
np.random.seed(seed)
np.random.randint(0,1000)

{'eval_metric': 'rmse', 'colsample_bylevel': 0.5, 'rate_drop': 0.5, 'seed': 713, 'tree_method': 'exact', 'booster': 'gbtree', 'colsample_bytree': 0.5, 'silent': 1, 'min_child_weight': 1000, 'subsample': 0.3, 'eta': 0.1, 'objective': 'reg:linear', 'max_depth': 3}


713

In [3]:
def preprocess(d, timelimit=[0,999999999], missing=0, index = []):
#     if(len(index) > 0):
#         index = index[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])]

#         dcp = deepcopy(d[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])])
#     del d
#     gc.collect()
    d.drop('timestamp', inplace=True, axis=1)
    d.fillna(missing, inplace=True)
    if(len(index) == 0):
        return d
    else:
        return d, index

In [4]:
## redo the training for train2, test2
dtr2, pred2, dtr2_index = loadHDF_train(datafilehead+'_train.h5',key='train2',columns = colnames)

In [6]:
print('import time limit (day): {0}, {1}'.format(dtr2['timestamp'].min()/unit, dtr2['timestamp'].max()/unit))

import time limit (day): 152.000021759, 181.999723495


In [7]:
## preprocess
dtr2, dtr2_index = preprocess(dtr2, timelimit = timelimit, missing=0, index=dtr2_index)
pred2 = preprocess(pred2, timelimit = timelimit, missing=0)

In [8]:
dte2, dte2_index = loadHDF_test(datafilehead+'_test.h5', key='test2', columns = colnames)
dte2 = preprocess(dte2, missing=0)
num_te2 = dte2.shape[0]
# dtest2 = xgb.DMatrix(dte2, missing=0)
# fmap = dte2.columns
# del dte2
# gc.collect()
# to be continued